# Extract Movie Info from TheMovieDB.org

In [1]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import pandas as pd
conn = http.client.HTTPSConnection("api.themoviedb.org")
source = 'themovedb'
payload = "{}"

apikey = 'd1fe56bd9e9956f57bb056a11cdbeb6d'

# Extract only the movies the actors were casted in 

In [2]:
path_to_json = './data/cast/themoviedb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

df = pd.concat([pd.read_json(f , dtype=str) for f in json_files], ignore_index = True)

In [3]:
movieDF = df[['movieID','movieTitle']]
movieDF.drop_duplicates(keep = False, inplace = True) 

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Define API to theMovieDB.org and export the movie to a json.

* Each Json filename is labeled with the <source name>_m_<movieID>.json
* Each json file will also tag the source of the data and the UTC create date
* We will use themoviedb.org ID as our own internal id for the movie.


In [4]:

def getMovieInfo(movieID):
    
    conn.request("GET", "/3/movie/{0}?language=en-US&api_key={1}".format(movieID, apikey), payload)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data

In [ ]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import logging
logger = logging.getLogger('movieInfo')
logger.setLevel(logging.INFO)

import pandas as pd
conn = http.client.HTTPSConnection("api.themoviedb.org")
source = 'themovedb'
payload = "{}"

apikey = 'd1fe56bd9e9956f57bb056a11cdbeb6d'

for index, row in movieDF.iterrows():

    try:
        _movieID = row['movieID']
        movieInfo = getMovieInfo(_movieID)
        release_date=  int( datetime.strptime(movieInfo['release_date'].strip(), '%Y-%m-%d').strftime('%Y%m%d'))
        if release_date>=20090601 and release_date <= 20190601:
            movieInfo.update({'_movieID':_movieID,'_source': source, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} )
            f = open('./data/movies/themoviedb/themoviedb_m_'+_movieID+'.json', 'w+')
            f.write(json.dumps(movieInfo))
            f.close()
        time.sleep(5)
    except Exception as e:
        logger.error('Failed to upload to ftp: '+ str(e))
        print('Error retriving movie ', row['movieTitle'])